In [12]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [13]:
# Root directory for dataset
dataroot = "/Volumes/gordonssd/cifar10"
# Directory to output images and model checkpoints
outf = "/Users/soymilk/Desktop/icr-gan/output"
# Number of workers for dataloader
workers = 2
# Batch size during training
batch_size = 64
# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 32
# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector (i.e. size of generator input)
nz = 100
# Size of feature maps in generator
ngf = 64
# Size of feature maps in discriminator
ndf = 64
# Number of training epochs
num_epochs = 5
# Learning rate for optimizers
lr = 0.0002
# Beta1 hyperparam for Adam optimizers
beta1 = 0.5
# Number of GPUs available. Use 0 for CPU mode.
ngpu = 0

<h3>Load CIFAR10</h3>

In [14]:
dataset = dset.CIFAR10(root=dataroot, download=True,
                       transform=transforms.Compose([
                           transforms.Resize(image_size),
                           transforms.ToTensor(),
                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
                       ]))
nc=3

dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

Files already downloaded and verified


<h3>Initialise pre-trained DCGAN</h3>

In [23]:
# load the models
from dcgan import Discriminator, Generator

netD = Discriminator(ngpu)
netG = Generator(ngpu)

# load weights
netD.load_state_dict(torch.load('weights/netD_epoch_4.pth', map_location=torch.device('cpu')))
netG.load_state_dict(torch.load('weights/netG_epoch_4.pth', map_location=torch.device('cpu')))
if torch.cuda.is_available():
    netD = netD.cuda()
    netG = netG.cuda()

In [24]:
# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

<h3>Loss function and Optimizers</h3>

In [25]:
criterion = nn.BCELoss()

fixed_noise = torch.randn(batch_size, nz, 1, 1, device=device)
real_label = 1
fake_label = 0

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

<h3>Train Balanced Consistency Regularization DCGAN (bCR-DCGAN)</h3>

In [26]:
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # Train with all-real batch
        netD.zero_grad()
        real_cpu = data[0].to(device)
        
        # bCR_TODO: Augment real images
        
        batch_size = real_cpu.size(0)
        label = torch.full((batch_size,), real_label, dtype=torch.float, device=device)

        output = netD(real_cpu)
        
        # bCR_TODO: Forward pass real w/ augmentation batch through D
        
        errD_real = criterion(output, label)
        errD_real.backward()
        D_x = output.mean().item()

        # train with fake
        noise = torch.randn(batch_size, nz, 1, 1, device=device)
        fake = netG(noise)
        
        # bCR_TODO: Augment generated images
        
        label.fill_(fake_label)
        output = netD(fake.detach())
        
        # bCR_TODO: Classify all fake w/ augmentation batch with D
        
        errD_fake = criterion(output, label)
        errD_fake.backward()
        D_G_z1 = output.mean().item()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        label.fill_(real_label)  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, label)
        errG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, num_epochs, i, len(dataloader),
                 errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % outf,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.detach(),
                    '%s/fake_samples_epoch_%03d.png' % (outf, epoch),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (outf, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (outf, epoch))

[0/5][0/782] Loss_D: 0.0286 Loss_G: 11.4995 D(x): 0.9999 D(G(z)): 0.0262 / 0.0000
[0/5][1/782] Loss_D: 0.3826 Loss_G: 6.3912 D(x): 0.7979 D(G(z)): 0.0167 / 0.0151
[0/5][2/782] Loss_D: 0.0063 Loss_G: 6.9444 D(x): 0.9989 D(G(z)): 0.0051 / 0.0055
[0/5][3/782] Loss_D: 0.0832 Loss_G: 7.5167 D(x): 0.9999 D(G(z)): 0.0486 / 0.0031
[0/5][4/782] Loss_D: 0.0091 Loss_G: 7.7433 D(x): 0.9997 D(G(z)): 0.0083 / 0.0030
[0/5][5/782] Loss_D: 0.0195 Loss_G: 7.5283 D(x): 0.9929 D(G(z)): 0.0118 / 0.0037
[0/5][6/782] Loss_D: 0.1141 Loss_G: 5.5922 D(x): 0.9151 D(G(z)): 0.0094 / 0.0156
[0/5][7/782] Loss_D: 0.0769 Loss_G: 9.0842 D(x): 0.9998 D(G(z)): 0.0627 / 0.0006
[0/5][8/782] Loss_D: 0.0233 Loss_G: 7.3721 D(x): 1.0000 D(G(z)): 0.0222 / 0.0034
[0/5][9/782] Loss_D: 0.0198 Loss_G: 8.0314 D(x): 0.9965 D(G(z)): 0.0142 / 0.0018
[0/5][10/782] Loss_D: 0.0331 Loss_G: 8.4379 D(x): 0.9997 D(G(z)): 0.0287 / 0.0012
[0/5][11/782] Loss_D: 0.0038 Loss_G: 8.5914 D(x): 0.9993 D(G(z)): 0.0031 / 0.0013
[0/5][12/782] Loss_D: 0.0

[0/5][100/782] Loss_D: 0.0118 Loss_G: 9.6646 D(x): 0.9904 D(G(z)): 0.0019 / 0.0012
[0/5][101/782] Loss_D: 0.0026 Loss_G: 9.4735 D(x): 0.9994 D(G(z)): 0.0020 / 0.0015
[0/5][102/782] Loss_D: 0.1648 Loss_G: 6.3785 D(x): 0.8970 D(G(z)): 0.0269 / 0.0176
[0/5][103/782] Loss_D: 0.0460 Loss_G: 7.1006 D(x): 0.9970 D(G(z)): 0.0377 / 0.0066
[0/5][104/782] Loss_D: 0.0711 Loss_G: 8.3423 D(x): 0.9994 D(G(z)): 0.0485 / 0.0020
[0/5][105/782] Loss_D: 0.0851 Loss_G: 8.0741 D(x): 0.9841 D(G(z)): 0.0525 / 0.0022
[0/5][106/782] Loss_D: 0.0227 Loss_G: 9.7038 D(x): 0.9831 D(G(z)): 0.0048 / 0.0008
[0/5][107/782] Loss_D: 0.1071 Loss_G: 6.2925 D(x): 0.9133 D(G(z)): 0.0020 / 0.0106
[0/5][108/782] Loss_D: 0.0286 Loss_G: 6.1847 D(x): 0.9823 D(G(z)): 0.0087 / 0.0134
[0/5][109/782] Loss_D: 0.1470 Loss_G: 9.7221 D(x): 0.9999 D(G(z)): 0.0923 / 0.0004
[0/5][110/782] Loss_D: 0.0138 Loss_G: 8.9512 D(x): 0.9977 D(G(z)): 0.0106 / 0.0019
[0/5][111/782] Loss_D: 0.0307 Loss_G: 7.6688 D(x): 0.9917 D(G(z)): 0.0206 / 0.0037
[0/5

[0/5][199/782] Loss_D: 0.0353 Loss_G: 6.9608 D(x): 0.9700 D(G(z)): 0.0022 / 0.0062
[0/5][200/782] Loss_D: 0.0264 Loss_G: 8.6795 D(x): 1.0000 D(G(z)): 0.0246 / 0.0008
[0/5][201/782] Loss_D: 0.0620 Loss_G: 9.0300 D(x): 0.9877 D(G(z)): 0.0318 / 0.0006
[0/5][202/782] Loss_D: 0.0372 Loss_G: 10.0501 D(x): 0.9896 D(G(z)): 0.0151 / 0.0004
[0/5][203/782] Loss_D: 0.0045 Loss_G: 8.1481 D(x): 0.9998 D(G(z)): 0.0042 / 0.0021
[0/5][204/782] Loss_D: 0.0041 Loss_G: 8.9351 D(x): 0.9971 D(G(z)): 0.0012 / 0.0008
[0/5][205/782] Loss_D: 0.0037 Loss_G: 10.3142 D(x): 0.9970 D(G(z)): 0.0006 / 0.0005
[0/5][206/782] Loss_D: 0.0020 Loss_G: 9.5900 D(x): 0.9988 D(G(z)): 0.0007 / 0.0007
[0/5][207/782] Loss_D: 0.0877 Loss_G: 6.6834 D(x): 0.9430 D(G(z)): 0.0081 / 0.0106
[0/5][208/782] Loss_D: 0.0416 Loss_G: 8.4056 D(x): 0.9962 D(G(z)): 0.0242 / 0.0011
[0/5][209/782] Loss_D: 0.1456 Loss_G: 9.6864 D(x): 0.9214 D(G(z)): 0.0236 / 0.0016
[0/5][210/782] Loss_D: 0.2278 Loss_G: 14.2875 D(x): 0.9894 D(G(z)): 0.1396 / 0.0000
[

[0/5][298/782] Loss_D: 0.1238 Loss_G: 9.8605 D(x): 0.9998 D(G(z)): 0.1000 / 0.0004
[0/5][299/782] Loss_D: 0.0907 Loss_G: 9.5600 D(x): 0.9996 D(G(z)): 0.0596 / 0.0003
[0/5][300/782] Loss_D: 0.0620 Loss_G: 8.1635 D(x): 0.9774 D(G(z)): 0.0267 / 0.0018
[0/5][301/782] Loss_D: 0.0215 Loss_G: 8.8303 D(x): 0.9903 D(G(z)): 0.0099 / 0.0015
[0/5][302/782] Loss_D: 0.0255 Loss_G: 7.9496 D(x): 0.9845 D(G(z)): 0.0083 / 0.0027
[0/5][303/782] Loss_D: 0.1410 Loss_G: 8.0965 D(x): 0.8928 D(G(z)): 0.0004 / 0.0040
[0/5][304/782] Loss_D: 0.1355 Loss_G: 8.6000 D(x): 0.9998 D(G(z)): 0.0654 / 0.0023
[0/5][305/782] Loss_D: 0.0088 Loss_G: 8.7306 D(x): 0.9989 D(G(z)): 0.0073 / 0.0040
[0/5][306/782] Loss_D: 0.0199 Loss_G: 9.1106 D(x): 0.9991 D(G(z)): 0.0159 / 0.0020
[0/5][307/782] Loss_D: 0.0401 Loss_G: 8.7277 D(x): 0.9688 D(G(z)): 0.0041 / 0.0034
[0/5][308/782] Loss_D: 0.0347 Loss_G: 8.4930 D(x): 0.9999 D(G(z)): 0.0294 / 0.0025
[0/5][309/782] Loss_D: 0.0093 Loss_G: 7.8345 D(x): 0.9990 D(G(z)): 0.0082 / 0.0030
[0/5

[0/5][397/782] Loss_D: 0.7446 Loss_G: 1.9369 D(x): 0.6451 D(G(z)): 0.0092 / 0.4120
[0/5][398/782] Loss_D: 0.5554 Loss_G: 15.6137 D(x): 1.0000 D(G(z)): 0.2669 / 0.0000
[0/5][399/782] Loss_D: 0.1379 Loss_G: 12.3466 D(x): 1.0000 D(G(z)): 0.0700 / 0.0000
[0/5][400/782] Loss_D: 0.0054 Loss_G: 11.2208 D(x): 0.9973 D(G(z)): 0.0026 / 0.0004
[0/5][401/782] Loss_D: 0.2310 Loss_G: 6.1137 D(x): 0.8404 D(G(z)): 0.0010 / 0.0192
[0/5][402/782] Loss_D: 0.0447 Loss_G: 5.4828 D(x): 0.9712 D(G(z)): 0.0068 / 0.0422
[0/5][403/782] Loss_D: 0.2074 Loss_G: 10.2847 D(x): 0.9837 D(G(z)): 0.1180 / 0.0002
[0/5][404/782] Loss_D: 0.1355 Loss_G: 11.1324 D(x): 0.9993 D(G(z)): 0.0495 / 0.0002
[0/5][405/782] Loss_D: 0.0209 Loss_G: 9.0078 D(x): 0.9851 D(G(z)): 0.0052 / 0.0020
[0/5][406/782] Loss_D: 0.0168 Loss_G: 8.8303 D(x): 0.9979 D(G(z)): 0.0122 / 0.0015
[0/5][407/782] Loss_D: 0.0133 Loss_G: 9.1442 D(x): 0.9905 D(G(z)): 0.0025 / 0.0018
[0/5][408/782] Loss_D: 0.2009 Loss_G: 8.0670 D(x): 0.8722 D(G(z)): 0.0007 / 0.0142

...

[4/5][344/782] Loss_D: 0.0527 Loss_G: 7.5052 D(x): 0.9989 D(G(z)): 0.0452 / 0.0024
[4/5][345/782] Loss_D: 0.0429 Loss_G: 7.3884 D(x): 0.9660 D(G(z)): 0.0040 / 0.0034
[4/5][346/782] Loss_D: 0.0076 Loss_G: 8.8309 D(x): 0.9955 D(G(z)): 0.0026 / 0.0022
[4/5][347/782] Loss_D: 0.0035 Loss_G: 7.5330 D(x): 0.9999 D(G(z)): 0.0034 / 0.0029
[4/5][348/782] Loss_D: 0.0198 Loss_G: 7.0743 D(x): 0.9999 D(G(z)): 0.0184 / 0.0042
[4/5][349/782] Loss_D: 0.0159 Loss_G: 7.5093 D(x): 0.9902 D(G(z)): 0.0058 / 0.0038
[4/5][350/782] Loss_D: 0.0314 Loss_G: 7.9768 D(x): 0.9858 D(G(z)): 0.0126 / 0.0022
[4/5][351/782] Loss_D: 0.0283 Loss_G: 6.8350 D(x): 0.9815 D(G(z)): 0.0083 / 0.0057
[4/5][352/782] Loss_D: 0.0156 Loss_G: 7.5515 D(x): 0.9887 D(G(z)): 0.0040 / 0.0039
[4/5][353/782] Loss_D: 0.0093 Loss_G: 8.1484 D(x): 0.9939 D(G(z)): 0.0029 / 0.0026
[4/5][354/782] Loss_D: 0.1208 Loss_G: 6.1562 D(x): 0.9339 D(G(z)): 0.0335 / 0.0092
[4/5][355/782] Loss_D: 0.0956 Loss_G: 8.0198 D(x): 0.9998 D(G(z)): 0.0622 / 0.0023
[4/5

[4/5][443/782] Loss_D: 0.0306 Loss_G: 7.2020 D(x): 0.9845 D(G(z)): 0.0140 / 0.0054
[4/5][444/782] Loss_D: 0.0102 Loss_G: 7.3650 D(x): 0.9977 D(G(z)): 0.0077 / 0.0041
[4/5][445/782] Loss_D: 0.0028 Loss_G: 7.9862 D(x): 0.9995 D(G(z)): 0.0022 / 0.0016
[4/5][446/782] Loss_D: 0.0107 Loss_G: 7.2469 D(x): 0.9988 D(G(z)): 0.0093 / 0.0041
[4/5][447/782] Loss_D: 0.0513 Loss_G: 7.2145 D(x): 0.9684 D(G(z)): 0.0106 / 0.0055
[4/5][448/782] Loss_D: 0.0085 Loss_G: 8.3090 D(x): 0.9938 D(G(z)): 0.0020 / 0.0022
[4/5][449/782] Loss_D: 0.0360 Loss_G: 6.6637 D(x): 0.9916 D(G(z)): 0.0251 / 0.0046
[4/5][450/782] Loss_D: 0.0052 Loss_G: 7.5476 D(x): 0.9987 D(G(z)): 0.0039 / 0.0026
[4/5][451/782] Loss_D: 0.0826 Loss_G: 7.0829 D(x): 0.9318 D(G(z)): 0.0025 / 0.0045
[4/5][452/782] Loss_D: 0.0238 Loss_G: 7.0455 D(x): 0.9935 D(G(z)): 0.0155 / 0.0056
[4/5][453/782] Loss_D: 0.0112 Loss_G: 6.5628 D(x): 1.0000 D(G(z)): 0.0109 / 0.0051
[4/5][454/782] Loss_D: 0.0775 Loss_G: 8.4351 D(x): 0.9924 D(G(z)): 0.0550 / 0.0017
[4/5

[4/5][542/782] Loss_D: 0.0180 Loss_G: 7.4698 D(x): 0.9998 D(G(z)): 0.0163 / 0.0028
[4/5][543/782] Loss_D: 0.0561 Loss_G: 7.1787 D(x): 0.9821 D(G(z)): 0.0313 / 0.0038
[4/5][544/782] Loss_D: 0.0220 Loss_G: 7.2045 D(x): 0.9934 D(G(z)): 0.0140 / 0.0031
[4/5][545/782] Loss_D: 0.0105 Loss_G: 8.1499 D(x): 0.9913 D(G(z)): 0.0015 / 0.0012
[4/5][546/782] Loss_D: 0.0178 Loss_G: 7.2868 D(x): 0.9916 D(G(z)): 0.0082 / 0.0037
[4/5][547/782] Loss_D: 0.0591 Loss_G: 6.2888 D(x): 0.9552 D(G(z)): 0.0095 / 0.0092
[4/5][548/782] Loss_D: 0.0204 Loss_G: 7.0979 D(x): 0.9991 D(G(z)): 0.0178 / 0.0062
[4/5][549/782] Loss_D: 0.0052 Loss_G: 7.9368 D(x): 0.9978 D(G(z)): 0.0030 / 0.0024
[4/5][550/782] Loss_D: 0.0030 Loss_G: 8.0443 D(x): 0.9995 D(G(z)): 0.0025 / 0.0020
[4/5][551/782] Loss_D: 0.0048 Loss_G: 8.1697 D(x): 0.9973 D(G(z)): 0.0021 / 0.0017
[4/5][552/782] Loss_D: 0.0046 Loss_G: 8.1712 D(x): 0.9993 D(G(z)): 0.0039 / 0.0028
[4/5][553/782] Loss_D: 0.0072 Loss_G: 7.5084 D(x): 0.9999 D(G(z)): 0.0070 / 0.0035
[4/5

[4/5][641/782] Loss_D: 0.0069 Loss_G: 8.5470 D(x): 0.9997 D(G(z)): 0.0063 / 0.0026
[4/5][642/782] Loss_D: 0.0072 Loss_G: 8.0148 D(x): 1.0000 D(G(z)): 0.0069 / 0.0026
[4/5][643/782] Loss_D: 0.0301 Loss_G: 8.8325 D(x): 0.9718 D(G(z)): 0.0006 / 0.0009
[4/5][644/782] Loss_D: 0.0164 Loss_G: 7.6289 D(x): 0.9993 D(G(z)): 0.0144 / 0.0039
[4/5][645/782] Loss_D: 0.0191 Loss_G: 7.4104 D(x): 0.9992 D(G(z)): 0.0169 / 0.0027
[4/5][646/782] Loss_D: 0.0113 Loss_G: 8.6266 D(x): 0.9948 D(G(z)): 0.0056 / 0.0020
[4/5][647/782] Loss_D: 0.1971 Loss_G: 7.2777 D(x): 0.8777 D(G(z)): 0.0011 / 0.0073
[4/5][648/782] Loss_D: 0.0208 Loss_G: 6.8867 D(x): 0.9991 D(G(z)): 0.0184 / 0.0089
[4/5][649/782] Loss_D: 0.0151 Loss_G: 6.8213 D(x): 1.0000 D(G(z)): 0.0143 / 0.0054
[4/5][650/782] Loss_D: 0.0206 Loss_G: 6.8701 D(x): 0.9987 D(G(z)): 0.0184 / 0.0030
[4/5][651/782] Loss_D: 0.1711 Loss_G: 10.5016 D(x): 0.9984 D(G(z)): 0.1011 / 0.0001
[4/5][652/782] Loss_D: 0.0051 Loss_G: 9.0872 D(x): 0.9977 D(G(z)): 0.0028 / 0.0007
[4/

[4/5][740/782] Loss_D: 0.0215 Loss_G: 8.4377 D(x): 1.0000 D(G(z)): 0.0181 / 0.0017
[4/5][741/782] Loss_D: 0.0055 Loss_G: 8.0688 D(x): 0.9992 D(G(z)): 0.0046 / 0.0018
[4/5][742/782] Loss_D: 0.0015 Loss_G: 9.6619 D(x): 0.9998 D(G(z)): 0.0013 / 0.0010
[4/5][743/782] Loss_D: 0.0023 Loss_G: 9.4941 D(x): 1.0000 D(G(z)): 0.0022 / 0.0009
[4/5][744/782] Loss_D: 0.0029 Loss_G: 9.8571 D(x): 0.9981 D(G(z)): 0.0010 / 0.0008
[4/5][745/782] Loss_D: 0.0132 Loss_G: 8.6027 D(x): 0.9887 D(G(z)): 0.0012 / 0.0013
[4/5][746/782] Loss_D: 0.0092 Loss_G: 9.3611 D(x): 0.9919 D(G(z)): 0.0010 / 0.0011
[4/5][747/782] Loss_D: 0.0042 Loss_G: 7.1844 D(x): 0.9999 D(G(z)): 0.0041 / 0.0033
[4/5][748/782] Loss_D: 0.0127 Loss_G: 8.0550 D(x): 0.9934 D(G(z)): 0.0059 / 0.0032
[4/5][749/782] Loss_D: 0.0278 Loss_G: 7.4930 D(x): 0.9990 D(G(z)): 0.0253 / 0.0031
[4/5][750/782] Loss_D: 0.0171 Loss_G: 9.9424 D(x): 1.0000 D(G(z)): 0.0111 / 0.0004
[4/5][751/782] Loss_D: 0.0115 Loss_G: 8.3621 D(x): 0.9965 D(G(z)): 0.0077 / 0.0030
[4/5